# Curso Análisis de Redes

## Cátedra Teoría de las Telecomunicaciones - Universidad ORT Uruguay



# Control de congestión en redes

In [1]:
#using Pkg;Pkg.instantiate(); Pkg.update()

## Introducción

Uno de los problemas fundamentales de las redes de telecomunicaciones es el *reparto o asignación de recursos*. Una primera aproximación a este problema lo vimos al estudiar el acceso al medio en una red local, donde un conjunto de nodos quiere compartir un único medio común.

**Preguntas:**

 * ¿Cómo se realiza el reparto de recursos en una red compleja o de mayor escala?
 
 * ¿Cómo logramos que dicho reparto sea *distribuido*, es decir que no requiera un controlador central?
 
 * ¿Cuál o cuáles son los repartos "justos" y "eficientes"?
 
Estas preguntas nos llevan a la noción de **control de congestión** en redes.

## Un poco de historia

<img style="float: right; width: 400px; padding-left: 2em" src="../images/pots.jpg"/>

La primera red compleja de telecomunicaciones fue la **red telefónica** tradicional.

* Basada en una única aplicación (voz)

* Tenía como objetivo que el receptor fuera *lo más simple posible* para garantizar su adopción.

* Inicialmente operada manualmente (cables), luego electromecánica y por último digitalizada.

* Aplicación homogénea: los recursos necesarios para una llamada consisten en un **circuito**

* En el caso digital, son $64kbps$ (una muestra de $8$ bits cada $125 \mu s$, frec. de muestreo $8kHz$ y ley PCM de codificación.



### La red telefónica: estrategia de diseño

Pensemos en el problema de interonectar $N$ personas o nodos. No tiene sentido poner un cable entre cada par de nodos, ya que se requieren:

$$C^N_2 = \frac{N(N-1)}{2}$$

cables (observar que crece como $N^2$), y no todos se usan al mismo tiempo (o no se usan nunca).

**Estrategia:**

<img style="width:400px; float:right; padding-left:1em" src="../images/hub.png" />

Construir *centrales telefónicas* que conecten los $N$ nodos a un punto central ($N$ cables).

 * Cada nodo conectado a la central.
 
 * En la central se conectan *solo las lineas necesarias* cuando se van a usar.
 
 * Inicialmente esto lo hacía la operadora con un cable, luego relays electromecánicos, y luego conmutadores digitales.
 
 
**Congestión:**

Habría congestión si no me alcanzan los cables en la central. Pero podemos diseñar para que esto no ocurra.

### La red telefónica: interconectando centrales

¿Qué ocurre si queremos hacer llamadas entre puntos distantes? Interconectamos las centrales:

![Dos centrales](../images/twohubs.png)

### La red telefónica: congestión

En este caso la congestión se produce si hay demasiadas llamadas "larga distancia" ente centrales.

Sin embargo no era necesaria la noción de "control de congestión" porque:

* Todas las llamadas ocupan un "circuito" (recursos conocidos).

* Podemos diseñar el enlace trunk de manera de que la cantidad de circuitos sea suficiente para que la probabilidad de bloqueo sea baja.

* Herramienta: fórmula de Erlang, teoría de colas, como ya vimos.


Siguiendo esta estrategia se construyó la red mundial PSTN (Public Switched Telephone Network) que interconectó todo el mundo, utilizando una *jerarquía* de centrales cada vez más grandes y circuitos SDH (Synchronous Digital Hierarchy) digitales.


## Las redes de datos

<img style="float:right; padding-left:2em; width:300px" src="../images/kleinrock.jpg"/>

Hacia los años 60 emerge un nuevo paradigma de comunicación: las **redes de paquetes** o **datagramas**.

 * Se busca una red que permita enviar cualquier tipo de dato.
 
 * Para eso, las terminales dividen los datos en "paquetes" o "datagramas" que son enviados por los enlaces.
 
 * Aparece el concepto de **enrutador** (router) que recibe los datagramas y los redirige hacia el destino.
 
 * Los paquetes de diferentes fuentes se mezclan en el enrutador y son atendidos en orden, logrando el reparto de recursos.

### Arquitectura

Arquitectura de una red de datagramas:

![Network](../images/network.png)

### Redes de datagramas: estrategia de diseño

Para diseñar e interconectar estas redes, hacia los años 80 emerge el *modelo de capas* que divide el problema de comunicación en varias capas, de alcance cada vez más global o extremo a extremo.

* **Capa física:** ligada a la modulación y al uso del medio.
* **Capa de enlace:** implementa la comunicación entre nodos *adyacentes*.
* **Capa de red:** provee un sustrato común de comunicación (ej: espacio de direcciones, rutas).
* **Capa de transporte:** primera capa extremo a extremo, regula la comunicación entre nodos distantes.
* **Capas de sesión, presentación, aplicación:** implementan los protocolos que soportan las aplicaciones.

### Modelo de capas: el stack TCP/IP

El stack de protocolos más exitoso de esta arquitectura es el de TCP/IP, base de la Internet actual.

![Modelo de capas](../images/tcpip_stack.png)

### La necesidad de control

En estas redes, que soportan múltiples aplicaciones, la mayoría tienen **requerimientos elásticos** de ancho de banda.

Esto es:

 * Tienen datos para transmitir (ej: un archivo).
 * Pueden hacer uso de todo el ancho de banda (velocidad de transmisión) disponible.
 * Idealmente buscan maximizar el *throughput* individualmente, es decir transferir lo más rápido posible.
 
**Ejemplo:** transferencia de archivos (HTTP, FTP, BitTorrent).

Otras aplicaciones tienen otros requerimientos:

 * Ej: llamadas de voz, baja latencia y jitter (variabilidad de retardo).
 * Ej: aplicaciones interctivas, baja latencia.

### El colapso de congestión

Hacia los años 80, el protocolo TCP (Transmission Control Protocol) estaba implementado:

* Permite *secuenciar* los datos enviados para garantizar su correcto ordenamiento en el receptor y su integridad.
* Provee mensajes de reconocimiento (ACK) para informar al emisor del estado de la transferencia.
* Mantiene ordenado el flujo de datos hacia las capas superiores.

Sin embargo, **no se especificaba cuántos datos se podían enviar simultáneamente**. Esto es, cada aplicación decidía la "ventana" de datos a enviar. A veces solo respondía a un *control de flujo* para no saturar al receptor.

**Colapso de congestión:**

Ocurre por primera vez en Internet en 1986, cuando el backbone de la National Science Foundation baja su velicidad efectiva de 32kbps a 40bps.

En ese momento aparece la noción de **control de congestión**. Se requiere un mecanismo de feedback a los transmisores para **no saturar la red**.

## Control de flujo mediante ventanas

La estrategia para regular la velocidad de transmisión en un protocolo como TCP consiste en utilizar una *ventana de transmisión*.

> **Ventana de transmisión**:
>
> Nos. de secuencia de los paquetes que puedo mantener "en viaje" sin haber recibido el ACK correspondiente.

**Ejemplo:** Protocolo stop-and-wait, espero el ACK antes de enviar el siguiente paquete. Si no llega retransmito el mismo.

 * Mantiene el orden.
 * Poco eficiente si el retardo es muy largo.
 * Corresponde a tomar ventana de transmisión $W=1$.

### Análisis de un protocolo de ventana fija.

Supongamos que tenemos el siguiente diagrama de un enlace cuello de botella, donde el transmisor utiliza un protocolo de ventana deslizante con tamaño fijo $W$. ¿Qué valor de $W$ conviene utilizar?

![Single link](../images/single_link.png)



### Diagrama temporal

Tomemos a modo de ejemplo $W=4$ y realicemos un diagrama temporal de los intercambios:

![Ventana deslizante](../images/ventana_deslizante.png)

### Observaciones

* En régimen, los paquetes no se mandan todos juntos. La cadencia de los ACK me da información sobre la velocidad de la red.
* Los ACK quedan separados un tiempo de envío $1/C$.
* Si la ventana es baja, se logran enviar todos los $W$ paquetes antes de que llegue el primer ACK de la ventana, y la red no se aprovecha completamente.

**Tiempo en envíos:** 
$$WT_{envio} = W/C.$$

**Tiempo en espera:** 
$$T_{total} - W T_{envio} = 1/C + d - W/C.$$

### Ventana óptima de transmisión

Para que el tiempo en espera sea $0$ y por consiguiente llenar la capacidad se requiere:

$$1/C + d - W^*/C = 0 \Rightarrow W^* = Cd + 1.$$

Como en general $Cd$ no es entero, en realidad tomamos $W^*=Cd$.

**Conclusión:** el no. de paquetes en viaje que optimiza la velocidad es igual al producto retardo por ancho de banda.

**Ejemplo:** enlace de 3Mbps con paquetes de 1500 bytes, es decir $C=250$ paq/s. Si el retardo de comunicación es de $d=200ms$, entonces $W^*=C d = 250 \text{ paq}/s \times 0.2 s = 50$ paquetes.

### Diagrama

![Ventana optima](../images/ventana_deslizante_2.png)

### Velocidad de transmisión

Si la ventana $W$ es menor que la ventana óptima, el ciclo en régimen es enviar $W$ paquetes cada $d$ segundos (despreciando el tiempo de envío de 1 paquete). Por lo tanto, la velocidad es:

$$W< W^* = Cd \Rightarrow x = \frac{W}{d}$$

y el tiempo que observo entre la salida de un paquete y la vuelta de su ACK se denomina Round Trip Time y es $RTT=d$.

Cuando la ventana llega a $W^*$, entonces:

$$W=W^* =Cd \Rightarrow x= \frac{Cd}{d}= C$$ 

y se utiliza completamente el ancho de banda disponible.

### Relación ventana y cola

Surge la pregunta: ¿podemos seguir aumentando la ventana más allá de $W^*$? ¿Aumenta la velocidad?

Tomemos $W=12$ en el diagrama anterior:

![Ventana deslizante y cola](../images/ventana_deslizante_3.png)


### Ventana deslizante y cola

Si $W>W^*$, los paquetes extra quedan encolados a la entrada del cuello de botella.

En régimen, la cola es:

$$q = W - W^* = W-Cd.$$

Por lo tanto, se incrementa el round trip time de un paquete en el retardo de cola:

$$RTT = d + q/C$$

Y la velocidad de transmisión es:

$$x=\frac{W}{RTT} = \frac{W}{d+q/C} = \frac{Cd+q}{d+q/C} = C.$$

Es decir, no aumenta más el throughput, solo aumenta el retardo de cola.

### Resumen: protocolo de ventana fija.

Resumiendo, las magnitudes de interés en un enlace tienen el siguiente comportamiento en función de la ventana:

![Bottleneck](../images/ventana_bottleneck.png)

### Limitación del buffer

En la práctica, el buffer de paquetes del cuello de botella no es infinito. Tendrá una limitación $B$. Por lo tanto, si la cola supera $B$ se perderán paquetes.
![Single link](../images/single_link.png)


**Máxima cantidad de paquetes "en vuelo":** $$W_{max} = W^* + B = Cd+B.$$

Si en una ventana se envía más de esta cantidad de paquetes, los sobrantes no podrán ser encolados y se perderán. Es responsabilidad del protocolo retransmitirlos.

## El control de congestión de TCP

Discutamos ahora una implementación práctica del control de congestión.

**Objetivos:**

 * Trabajar por encima de la ventana óptima de transmisión $W^*$ para aumentar el throughput.
 
 * Evitar el retardo de cola y las pérdidas de paquetes (en particular, trabajar por debajo de $W_{max}$).
 
**Problemas:**

 * El emisor no conoce dónde está ni qué capacidad tiene el cuello de botella.
 
 * Tampoco sabe el retardo de la comunicación.
 
 * Además, no sabe si no comparte esa ventana con otras conexiones.
 
**Idea:**

 * Realizar un control en feedback, aumentando la ventana hasta saturar y realimentando cuando veamos indicaciones de *congestión*.

### TCP - Tahoe

<img style="float:right; padding-left:2em" src="../images/tcp_tahoe.jpg"/>

Fue el primer mecanismo de control de congestión, propuesto por Van Jacobson en 1988. Consta de dos fases:


**Fase Slow-Start:**
 * Se inicia la ventana $cwnd=1$ MSS (maximum segment size).
 * Por cada ACK recibido, se incrementa $cwnd \to cwnd+1$.
 * Al cabo de una ventana transmitida correctamente, $cwnd$ se duplicó,
 
**Fase Congestion-Avoidance:**

 * Se inicia al llegar a un umbral $cwnd=ssthresh$ (slow start threshold)
 * Se lleva la cuenta de ACKs recibidos. Si la ventana es $cwnd$, al recibir $cwnd$ ACKs correctamente, se incrementa $cwnd\to cwnd+1$.
 * Esto hace que al cabo de una ventana transmitida correctamente, $cwnd$ aumenta en $1$. 

**Pérdidas:**

 * Se detectan por timeout o triple ACK duplicado. Se interpretan como señal de congestión.
 * Se actualiza $ssthresh \to cwnd/2$.
 * Se vuelve a $cwnd=1$ y fase slow start. 

### TCP - Reno

<img style="float:right; padding-left:2em; width:400px" src="../images/tcp_reno.png"/>

Fue una actualización del anterior, presentada en 1990. Agrega mecansimsos de recuperación ante pérdidas. Desde el punto de vista del control de congestión el único cambio significativo es el siguiente:

**Fase Slow-Start y Fase Congestion Avoidance:** sin cambios.

**Pérdidas:**

 * Se detectan por timeout o triple ACK duplicado. Se interpretan como señal de congestión.
 * Se actualiza $ssthresh \to cwnd/2$.
 * Si es triple ACK duplicado (caso más frecuente), se vuelve a $cwnd=ssthresh$ y continúo en CA.
 * Si es timeout, se vuelve a $cwnd=1$ y fase SS.

## Ejercicio

Considere un enlace cuello de botella de capacidad $100$ paquetes por segundo y retardo de ida y vuelta $d=100ms$. Suponga que el buffer a la entrada del enlace admite hasta $5$ paquetes.

1. Calcule la ventana óptima de transmisión y la cantidad máxima de paquetes en vuelo.
2. Dibuje la evolución de la ventana de TCP/Reno arrancando de $W=1$ hasta que llega a régimen, incluyendo las fases slow-start y congestion-avoidance. Suponga que el ssthresh es 40 paquetes (aprox. 64K en implementaciones reales).
3. ¿Se logra trabajar 100% del tiempo a la máxima velocidad?
4. ¿Cuál es la probabilidad de pérdida de paquetes en régimen?